# Example source seperation of a trumpet with Open Unmix
We provide an example of the source seperation of a trumpet. The result shows that the model learns a meaningful task. However, the model complexity does not seem to be sufficiently complex to seperate the trumpet from other wind instruments. This tasks is harder than for example seperating voice from drum.
We observe even less qualitative output for French horn or guitar because there sound is similar to other instruments in the considered ensembles. Moreover, they have longer pauses leading to a more inbalanced dataset.

In [1]:
#import scipy.signal
print('Start')
from IPython.display import Audio, display
import numpy as np
import tqdm
import matplotlib.pyplot as plt
import warnings
import random
import sklearn.preprocessing
import norbert
import torch
import os
import pickle 

warnings.simplefilter(action='ignore', category=FutureWarning)

import time
import torch.nn as nn
import torch.optim as optim

import openunmix

from dataloader_slakh import SlakhDataset

%load_ext autoreload
%autoreload 2

Start


Initialize the dataset

In [2]:
%cd ../open-unmix-pytorch/
%pwd

C:\Users\User\Documents\GitHub\uni\ddspzart\ws_2\open-unmix-pytorch


'C:\\Users\\User\\Documents\\GitHub\\uni\\ddspzart\\ws_2\\open-unmix-pytorch'

## Set up the model

In [12]:
SAVE_PATH = f"../source_seperation/data/checkpoints/"
CKPT_FILE = "exp4_trumpet.pt"
DATASET_STAT_FILE = "exp4_dataset_statistics.pickle" 
%pwd

'C:\\Users\\User\\Documents\\GitHub\\uni\\ddspzart\\ws_2\\open-unmix-pytorch'

In [5]:
with open(SAVE_PATH+DATASET_STAT_FILE, 'rb') as targets_file:
    stat = pickle.load(targets_file)
    mean, scale = stat

In [13]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(42)

device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
device = "cpu"

unmix = openunmix.model.OpenUnmix(
    input_mean=mean,
    input_scale=scale,
    nb_channels=1,
    hidden_size=512,
    max_bin=512,
    nb_bins=2048+1
).to(device)

optimizer = optim.RMSprop(unmix.parameters(), lr=0.005)
criterion = torch.nn.MSELoss()


# Demonstrate the model 

In [20]:
# Testing -> Here: Just use one sample.
# We do not provide the whole data set here. For training, the whole SLAKH dataset was used.
test_dataset = SlakhDataset(split='test',seq_duration=5.0) 
test_sampler = torch.utils.data.DataLoader(test_dataset, batch_size=8)

track = test_dataset[0]

In [21]:
# Filter the instrument (here Trumpet)
test_dataset.target = 'Trumpet'
test_dataset.filter_target()

In [14]:
# Load the trained model (here the trumpet source seperation)
checkpoint = torch.load(SAVE_PATH+CKPT_FILE)
unmix.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [31]:
# Seperate with customly trained models

audio_torch = track[0][None,...].float().to(device)

# Here, we  only apply one source seperation model to demonstrate that an application of openunmix is in general possible too for Wind instruments.
# However, a more thorough neural network architecture optimization had to be performed for achieving better results.
target_models = {"Trumpet": unmix} 
own_seperator = openunmix.model.Separator(target_models,nb_channels=1).to(device)
y = track[1][None,...].float().to(device).squeeze()

y_hat = own_seperator.forward(audio_torch).clone().detach().squeeze()

display(Audio(track[0], rate=44100))
display(Audio(y.cpu().numpy(), rate=44100))
display(Audio(y_hat.cpu().numpy(), rate=44100))


We provide testing functionality in the notebook "evaluate_openunmix_seperator.ipynb". Here, we do not to provide a thorough discussion of the training process even though this needed time and effort also including the usage of the EPFL cluster.
However, we think that this does not deliver key findings for the CM class.